# 7 WorkFlow

### 7.1 Objetivo

Presentar un workflow/pipeline completo al que los estudiantes deberán enriquecer

#### 7.2  Seteo del ambiente en Google Colab

Esta parte se debe correr con el runtime en Python3
<br>Ir al menu, Runtime -> Change Runtime Type -> Runtime type ->  **Python 3**

Conectar la virtual machine donde esta corriendo Google Colab con el  Google Drive, para poder tener persistencia de archivos

In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Para correr la siguiente celda es fundamental en Arranque en Frio haber copiado el archivo kaggle.json al Google Drive, en la carpeta indicada en el instructivo

<br>los siguientes comando estan en shell script de Linux
*   Crear las carpetas en el Google Drive
*   "instalar" el archivo kaggle.json desde el Google Drive a la virtual machine para que pueda ser utilizado por la libreria  kaggle de Python
*   Bajar el  **dataset_pequeno**  al  Google Drive  y tambien al disco local de la virtual machine que esta corriendo Google Colab
*   Bajar el **dataset_historico** al Google Drive y tambien al disco local de la virtual machine que esta corriendo Google Colab



*  Si usted eligió modalidad *Gerencial* entonces NO debe modificar nada, su liderazgo logró que otras trabajaron por usted
*  Si usted eligió modalidad de *Analista Junior*, entonces donde dice archivo="gerencial_competencia_2025.csv.gz"  lo debe cambiar por  archivo="analistajr_competencia_2025.csv.gz"
*  Si usted eligió modalidad *Analista Senior*,  ya se las ingenierá SIN preguntar !

In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets


webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/labo1r"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"


archivo="dataset_pequeno.csv"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

#-------

archivo="gerencial_competencia_2025.csv.gz"

if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi


if ! test -f $destino_local/$pequeno; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi


## 7.3  Workflow

## Inicializacion

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

limpio el ambiente de R

In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Mon Jun 30 11:20:54 2025"

In [2]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,653516,35.0,1439405,76.9,1439405,76.9
Vcells,1218045,9.3,8388608,64.0,1924961,14.7


In [3]:
require("data.table")

if( !require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


Attaching package: ‘R.utils’


The following object is masked from ‘package:utils’:

    timestamp


The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, isOpen, nullfile, parse, use, warnings




#### Parametros
Si es gerente, no cambie nada
<br>Si es Analista, cambie el nombre del dataset

In [ ]:
PARAM <- list()
PARAM$semilla_primigenia <- 777743

PARAM$experimento <- "byo 50 - Completo - Gen5 - lag"
PARAM$dataset <- "analistasr_competencia_2025.csv.gz"

#### Carpeta del Experimento

In [5]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

### 7.3.1   Preprocesamiento del dataset

#### 7.3.1.1  DT incorporar dataset

In [6]:
# lectura del dataset
dataset <- fread(paste0("/content/buckets/b1/datasets/", PARAM$dataset))

#### 7.3.1.2  CA  Catastrophe Analysis
Se intentan reparar las variables que para un mes están con todos los valores en cero.

El método que se utiliza es **Machine Learning** se asigna NA also valores, si ha leido bien, es la "anti imputación de valores faltantes"
<br> Usted podrá aplicar aquí otros métodos

In [7]:
dataset[ foto_mes==202006, internet:=NA]
dataset[ foto_mes==202006, mrentabilidad:=NA]
dataset[ foto_mes==202006, mrentabilidad_annual:=NA]
dataset[ foto_mes==202006, mcomisiones:=NA]
dataset[ foto_mes==202006, mactivos_margen:=NA]
dataset[ foto_mes==202006, mpasivos_margen:=NA]
dataset[ foto_mes==202006, mcuentas_saldo:=NA]
dataset[ foto_mes==202006, ctarjeta_visa_transacciones:=NA]
dataset[ foto_mes==202006, mtarjeta_visa_consumo:=NA]
dataset[ foto_mes==202006, mtarjeta_master_consumo:=NA]
dataset[ foto_mes==202006, ccallcenter_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]
dataset[ foto_mes==202006, chomebanking_transacciones:=NA]

#### 7.3.1.3  DR  Data Drifting
Se intenta corregir el data drifting, quizas ajustando por IPC ...
<br>Esta parte podrá ser abordada por todos los Analistas y también la Gerenciapero se decide pedagogicamente no incluirla en esta primer version para reducir la carga cognitiva

In [8]:
# sin codigo en esta primera version del workflow

#### 7.3.1.3  FE_intra_manual Feature Engineering intra-mes

Agrego campos nuevos dentro del mismo mes, SIN considerar la historia.

In [9]:
# esta funcion atributos presentes existe debido a que las modalidades poseen datasets con distinta cantidad de campos
atributos_presentes <- function( patributos )
{
  atributos <- unique( patributos )
  comun <- intersect( atributos, colnames(dataset) )

  return(  length( atributos ) == length( comun ) )
}

# el mes 1,2, ..12
if( atributos_presentes( c("foto_mes") ))
  dataset[, kmes := foto_mes %% 100]

# variable extraida de una tesis de maestria de Irlanda
if( atributos_presentes( c("mpayroll", "cliente_edad") ))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]


In [10]:
# FE Manual by Fede y Chino
dataset <- dataset[, nueva1 := ctrx_quarter*mcaja_ahorro] # Producto de las 2 variables más importantes
dataset <- dataset[, nueva2 := ifelse(Visa_mlimitecompra == 0, NA, Visa_mconsumototal/Visa_mlimitecompra)] # Consumo relativo de la tarjeta visa
dataset <- dataset[, nueva3 := ifelse(Master_mlimitecompra == 0, NA, Master_mconsumototal/Master_mlimitecompra)] # Consumo relativo de la Master
dataset <- dataset[, nueva4 := ifelse(Visa_cconsumos == 0, 0, Visa_mconsumototal/Visa_cconsumos)] # Consumo promedio visa
dataset <- dataset[, nueva5 := ifelse(Master_cconsumos == 0, 0, Master_mconsumototal/Master_cconsumos)] # Consumo promedio master
dataset <- dataset[, nueva6 := mtransferencias_recibidas-mtransferencias_emitidas] # Diferencia entre entradas y salidas de dinero
dataset <- dataset[, nueva7 := (cplazo_fijo+cseguro_vida+cseguro_auto+cseguro_vivienda+cseguro_accidentes_personales+ccaja_seguridad)/cproductos] # Proporción de productos más "estables" contratados
dataset <- dataset[, nueva8 := ifelse(mcuentas_saldo == 0, NA, (mprestamos_personales+mprestamos_prendarios+mprestamos_hipotecarios)/mcuentas_saldo)] # Relación prestamos con el saldo
dataset <- dataset[, nueva9 := ifelse((is.na(mtarjeta_visa_consumo) | mtarjeta_visa_consumo == 0) & (is.na(mtarjeta_master_consumo) | mtarjeta_master_consumo == 0), NA,
                                      ifelse(is.na(mtarjeta_visa_consumo), mtarjeta_master_descuentos/mtarjeta_master_consumo,
                                              ifelse(is.na(mtarjeta_master_consumo), mtarjeta_visa_descuentos/mtarjeta_visa_consumo, (mtarjeta_visa_descuentos+mtarjeta_master_descuentos)/(mtarjeta_visa_consumo+mtarjeta_master_consumo))))] # Porcentaje de descuento sobre total de consumo
dataset <- dataset[, nueva10 := ctrx_quarter/cliente_antiguedad] # Transacciones en el trimestre sobre antiguedad

dataset <- dataset[, nueva11 := (ctarjeta_debito_transacciones +
      ctarjeta_visa_transacciones + ctarjeta_master_transacciones + ccallcenter_transacciones +
      chomebanking_transacciones + ccajas_transacciones)] # cantidad total de transacciones

dataset <- dataset[, nueva12 := ifelse((ccajeros_propios_descuentos + ctarjeta_visa_descuentos + ctarjeta_master_descuentos) == 0,
                                        0,(mcajeros_propios_descuentos + mtarjeta_visa_descuentos + mtarjeta_master_descuentos) /
                                        (ccajeros_propios_descuentos + ctarjeta_visa_descuentos + ctarjeta_master_descuentos))]  # Descuentos por op_desc

dataset <- dataset[, nueva13 := ifelse(nueva11 ==0, 0,mcomisiones_mantenimiento / nueva11)] #Costo de mantenimiento por transacción
dataset <- dataset[, nueva14 := ifelse(nueva11 ==0, 0,mcomisiones_otras / nueva11)] #Costo de comisiones extra por transacción

dataset <- dataset[, nueva15 := mforex_buy + mforex_sell] #Monto transacciones forex
dataset <- dataset[, nueva16 := cforex_buy + cforex_sell] #cantidad transacciones forex
dataset <- dataset[, nueva17 := ifelse(nueva16 == 0, NA,nueva15 / nueva16)] #Monto por operación forex

dataset <- dataset[, nueva18 := ifelse(is.na(Master_delinquency) & is.na(Visa_delinquency), NA, pmax(Master_delinquency, Visa_delinquency, na.rm = TRUE))] # sufrió delincuencia


In [ ]:
# Top 20 impo
# gen_5_n_15	
dataset <- dataset[, gen_5_n_15 := (mtarjeta_visa_consumo+mpayroll_sobre_edad) ]

# gen_5_n_11	
dataset <- dataset[, gen_5_n_11 := (mprestamos_personales/ctarjeta_visa)]

# gen_3_n_2	
dataset <- dataset[, gen_3_n_2 := ((cpayroll_trx/cliente_edad)/mpagomiscuentas)]

# gen_4_n_39	
dataset <- dataset[, gen_4_n_39 := (mtarjeta_visa_consumo*(cliente_edad+catm_trx)) ]

# gen_1_n_28	
dataset <- dataset[, gen_1_n_28 := (mcuentas_saldo-mtarjeta_master_descuentos)]

# gen_3_n_26	
dataset <- dataset[, gen_3_n_26 := (mtarjeta_visa_consumo/(Visa_status/kmes))]

# gen_1_n_10	
dataset <- dataset[, gen_1_n_10 := (cliente_antiguedad*mcuentas_saldo)]

# gen_5_n_32	
dataset <- dataset[, gen_5_n_32 := ((cpayroll_trx/cliente_edad)+cprestamos_personales)]

# gen_3_n_15	
dataset <- dataset[, gen_3_n_15 := (mcuenta_corriente/(mcomisiones_otras+mcheques_depositados))]

# gen_5_n_27	
dataset <- dataset[, gen_5_n_27 := ((cdescubierto_preacordado/kmes)/((cdescubierto_preacordado/kmes)+Visa_delinquency))]

# gen_4_n_29	
dataset <- dataset[, gen_4_n_29 := ((mcuentas_saldo-mtarjeta_master_descuentos)/(thomebanking+ctransferencias_emitidas))]

# gen_3_n_7	
dataset <- dataset[, gen_3_n_7 := (mpayroll_sobre_edad/Visa_mfinanciacion_limite)]

# gen_2_n_18	
dataset <- dataset[, gen_2_n_18 := (cpayroll_trx/cliente_edad)]

# gen_3_n_27	
dataset <- dataset[, gen_3_n_27 := (((Master_mpagominimo+mpayroll_sobre_edad)/cliente_edad)/cplazo_fijo)]

# gen_5_n_34	
dataset <- dataset[, gen_5_n_34 := (Visa_status/(chomebanking_transacciones/kmes))]

# gen_3_n_29	
dataset <- dataset[, gen_3_n_29 := (ctarjeta_visa_transacciones/ctarjeta_visa)]

# gen_2_n_33	
dataset <- dataset[, gen_2_n_33 := ((Master_mpagominimo+mpayroll_sobre_edad)/cliente_edad)]

# gen_1_n_23	
dataset <- dataset[, gen_1_n_23 := (Master_mpagominimo+mpayroll_sobre_edad)]

# gen_2_n_2	
dataset <- dataset[, gen_2_n_2 := ((cdescubierto_preacordado/kmes)/internet)]

# gen_4_n_31	
dataset <- dataset[, gen_4_n_31 := (cliente_edad-((Visa_status/kmes)+(ctarjeta_master-ccajas_otras)))]

In [ ]:
# Top 20 impo #old
#gen_1_n_8
# dataset <- dataset[, gen_1_n_8 := (mpagomiscuentas+ctrx_quarter)]

# #gen_2_n_5
# dataset <- dataset[, gen_2_n_5 := (ctrx_quarter-ctarjeta_master_transacciones)]

# # gen_1_n_28
# dataset <- dataset[, gen_1_n_28 := (mcuentas_saldo-mtarjeta_master_descuentos)]

# # gen_3_n_28
# dataset <- dataset[, gen_3_n_28 := (cpayroll_trx*cproductos)]

# # gen_2_n_2
# dataset <- dataset[, gen_2_n_2 := (mcuenta_corriente/mcuenta_debitos_automaticos)]

# # gen_3_n_29
# dataset <- dataset[, gen_3_n_29 := (mtarjeta_visa_consumo/ctarjeta_visa)]

# # gen_3_n_3
# dataset <- dataset[, gen_3_n_3 := (cprestamos_personales/ctarjeta_debito_transacciones)]

# # gen_1_n_23
# dataset <- dataset[, gen_1_n_23 := (Master_mpagominimo+mpayroll_sobre_edad)]

# # gen_1_n_26
# dataset <- dataset[, gen_1_n_26 := (ctarjeta_visa_transacciones+matm)]

# # gen_1_n_10
# dataset <- dataset[, gen_1_n_10 := (cliente_antiguedad+mcuentas_saldo)]

# # gen_1_n_5
# dataset <- dataset[, gen_1_n_5 := (cpagodeservicios/Master_status)]

# # gen_3_n_7
# dataset <- dataset[, gen_3_n_7 := (mpayroll_sobre_edad/(internet+mprestamos_hipotecarios))]

# # gen_3_n_13
# dataset <- dataset[, gen_3_n_13 := (mcuentas_saldo-Master_msaldototal)]

# # gen_3_n_15
# dataset <- dataset[, gen_3_n_15 := (mcuentas_saldo/(cdescubierto_preacordado-ccajas_consultas))]

# # gen_3_n_30
# dataset <- dataset[, gen_3_n_30 := ((mcuenta_corriente/mcuenta_debitos_automaticos)+ctarjeta_master)]

# # gen_3_n_5
# dataset <- dataset[, gen_3_n_5 := (mrentabilidad_annual-(cinversion2-Visa_msaldopesos))]

# # gen_2_n_21
# dataset <- dataset[, gen_2_n_21 := ((cinversion2-Visa_msaldopesos)-kmes)]

# # gen_1_n_16
# dataset <- dataset[, gen_1_n_8 := (mprestamos_personales+Master_fultimo_cierre)]

# # gen_2_n_17
# dataset <- dataset[, gen_2_n_17 := (cdescubierto_preacordado-ccajas_consultas)]

# # gen_3_n_2
# dataset <- dataset[, gen_3_n_2 := ((ctrx_quarter-ctarjeta_master_transacciones)-(mactivos_margen+ccajas_otras))]


In [12]:
# visualizo las columas del dataset a esta etapa
colnames(dataset)

[1] "numero_de_cliente"                   
  [2] "foto_mes"                            
  [3] "active_quarter"                      
  [4] "cliente_vip"                         
  [5] "internet"                            
  [6] "cliente_edad"                        
  [7] "cliente_antiguedad"                  
  [8] "mrentabilidad"                       
  [9] "mrentabilidad_annual"                
 [10] "mcomisiones"                         
 [11] "mactivos_margen"                     
 [12] "mpasivos_margen"                     
 [13] "cproductos"                          
 [14] "tcuentas"                            
 [15] "ccuenta_corriente"                   
 [16] "mcuenta_corriente_adicional"         
 [17] "mcuenta_corriente"                   
 [18] "ccaja_ahorro"                        
 [19] "mcaja_ahorro"                        
 [20] "mcaja_ahorro_adicional"              
 [21] "mcaja_ahorro_dolares"                
 [22] "cdescubierto_preacordado"            
 [23] "mcuentas_saldo"                      
 [24] "ctarjeta_debito"                     
 [25] "ctarjeta_debito_transacciones"       
 [26] "mautoservicio"                       
 [27] "ctarjeta_visa"                       
 [28] "ctarjeta_visa_transacciones"         
 [29] "mtarjeta_visa_consumo"               
 [30] "ctarjeta_master"                     
 [31] "ctarjeta_master_transacciones"       
 [32] "mtarjeta_master_consumo"             
 [33] "cprestamos_personales"               
 [34] "mprestamos_personales"               
 [35] "cprestamos_prendarios"               
 [36] "mprestamos_prendarios"               
 [37] "cprestamos_hipotecarios"             
 [38] "mprestamos_hipotecarios"             
 [39] "cplazo_fijo"                         
 [40] "mplazo_fijo_dolares"                 
 [41] "mplazo_fijo_pesos"                   
 [42] "cinversion1"                         
 [43] "minversion1_pesos"                   
 [44] "minversion1_dolares"                 
 [45] "cinversion2"                         
 [46] "minversion2"                         
 [47] "cseguro_vida"                        
 [48] "cseguro_auto"                        
 [49] "cseguro_vivienda"                    
 [50] "cseguro_accidentes_personales"       
 [51] "ccaja_seguridad"                     
 [52] "cpayroll_trx"                        
 [53] "mpayroll"                            
 [54] "mpayroll2"                           
 [55] "cpayroll2_trx"                       
 [56] "ccuenta_debitos_automaticos"         
 [57] "mcuenta_debitos_automaticos"         
 [58] "ctarjeta_visa_debitos_automaticos"   
 [59] "mttarjeta_visa_debitos_automaticos"  
 [60] "ctarjeta_master_debitos_automaticos" 
 [61] "mttarjeta_master_debitos_automaticos"
 [62] "cpagodeservicios"                    
 [63] "mpagodeservicios"                    
 [64] "cpagomiscuentas"                     
 [65] "mpagomiscuentas"                     
 [66] "ccajeros_propios_descuentos"         
 [67] "mcajeros_propios_descuentos"         
 [68] "ctarjeta_visa_descuentos"            
 [69] "mtarjeta_visa_descuentos"            
 [70] "ctarjeta_master_descuentos"          
 [71] "mtarjeta_master_descuentos"          
 [72] "ccomisiones_mantenimiento"           
 [73] "mcomisiones_mantenimiento"           
 [74] "ccomisiones_otras"                   
 [75] "mcomisiones_otras"                   
 [76] "cforex"                              
 [77] "cforex_buy"                          
 [78] "mforex_buy"                          
 [79] "cforex_sell"                         
 [80] "mforex_sell"                         
 [81] "ctransferencias_recibidas"           
 [82] "mtransferencias_recibidas"           
 [83] "ctransferencias_emitidas"            
 [84] "mtransferencias_emitidas"            
 [85] "cextraccion_autoservicio"            
 [86] "mextraccion_autoservicio"            
 [87] "ccheques_depositados"                
 [88] "mcheques_depositados"                
 [89] "ccheques_emitidos"                 

#### 7.3.1.4  FE_rf Feature Engineering de nuevas variables a partir de hojas de Random Forest

Esto se mostrará unicamente a la *modalidad Analista Sr*

In [13]:
# No se implementa Feature Engineering a partir de Random Forest

#### 7.3.1.5  FEhist Feature Engineering historico

El Fature Engineering Histórico es la etapa que más aporta a la ganancia final, ya que enriquece cada registro del dataset con su historia.

Para cada campo del dataset original (*)
se crean lo siguientes campos de a partir de la historia
* lag1  lags de orden 1
* delta1  =  valor actual - lag1
* lag2  lags de orden 2
* delta2  = valor actual - lag2


(*) Excepto para los campos  <numero_de_cliente,  foto_mes,  clase_ternaria>

In [14]:
# Feature Engineering Historico

# todo es lagueable, menos la primary key y la clase
cols_lagueables <- copy( setdiff(
    colnames(dataset),
    c("numero_de_cliente", "foto_mes", "clase_ternaria")
) )

# https://rdrr.io/cran/data.table/man/shift.html

# lags de orden 1
dataset[,
    paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# lags de orden 2
dataset[,
    paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
    by = numero_de_cliente,
    .SDcols = cols_lagueables
]

# agrego los delta lags
for (vcol in cols_lagueables)
{
    dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
    dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}


Verificacion de los campos recien creados

In [15]:
ncol(dataset)
colnames(dataset)

[1] 958

[1] "numero_de_cliente"                          
  [2] "foto_mes"                                   
  [3] "active_quarter"                             
  [4] "cliente_vip"                                
  [5] "internet"                                   
  [6] "cliente_edad"                               
  [7] "cliente_antiguedad"                         
  [8] "mrentabilidad"                              
  [9] "mrentabilidad_annual"                       
 [10] "mcomisiones"                                
 [11] "mactivos_margen"                            
 [12] "mpasivos_margen"                            
 [13] "cproductos"                                 
 [14] "tcuentas"                                   
 [15] "ccuenta_corriente"                          
 [16] "mcuenta_corriente_adicional"                
 [17] "mcuenta_corriente"                          
 [18] "ccaja_ahorro"                               
 [19] "mcaja_ahorro"                               
 [20] "mcaja_ahorro_adicional"                     
 [21] "mcaja_ahorro_dolares"                       
 [22] "cdescubierto_preacordado"                   
 [23] "mcuentas_saldo"                             
 [24] "ctarjeta_debito"                            
 [25] "ctarjeta_debito_transacciones"              
 [26] "mautoservicio"                              
 [27] "ctarjeta_visa"                              
 [28] "ctarjeta_visa_transacciones"                
 [29] "mtarjeta_visa_consumo"                      
 [30] "ctarjeta_master"                            
 [31] "ctarjeta_master_transacciones"              
 [32] "mtarjeta_master_consumo"                    
 [33] "cprestamos_personales"                      
 [34] "mprestamos_personales"                      
 [35] "cprestamos_prendarios"                      
 [36] "mprestamos_prendarios"                      
 [37] "cprestamos_hipotecarios"                    
 [38] "mprestamos_hipotecarios"                    
 [39] "cplazo_fijo"                                
 [40] "mplazo_fijo_dolares"                        
 [41] "mplazo_fijo_pesos"                          
 [42] "cinversion1"                                
 [43] "minversion1_pesos"                          
 [44] "minversion1_dolares"                        
 [45] "cinversion2"                                
 [46] "minversion2"                                
 [47] "cseguro_vida"                               
 [48] "cseguro_auto"                               
 [49] "cseguro_vivienda"                           
 [50] "cseguro_accidentes_personales"              
 [51] "ccaja_seguridad"                            
 [52] "cpayroll_trx"                               
 [53] "mpayroll"                                   
 [54] "mpayroll2"                                  
 [55] "cpayroll2_trx"                              
 [56] "ccuenta_debitos_automaticos"                
 [57] "mcuenta_debitos_automaticos"                
 [58] "ctarjeta_visa_debitos_automaticos"          
 [59] "mttarjeta_visa_debitos_automaticos"         
 [60] "ctarjeta_master_debitos_automaticos"        
 [61] "mttarjeta_master_debitos_automaticos"       
 [62] "cpagodeservicios"                           
 [63] "mpagodeservicios"                           
 [64] "cpagomiscuentas"                            
 [65] "mpagomiscuentas"                            
 [66] "ccajeros_propios_descuentos"                
 [67] "mcajeros_propios_descuentos"                
 [68] "ctarjeta_visa_descuentos"                   
 [69] "mtarjeta_visa_descuentos"                   
 [70] "ctarjeta_master_descuentos"                 
 [71] "mtarjeta_master_descuentos"                 
 [72] "ccomisiones_mantenimiento"                  
 [73] "mcomisiones_mantenimiento"                  
 [74] "ccomisiones_otras"                          
 [75] "mcomisiones_otras"                          
 [76] "cforex"                                     
 [77] "cforex_buy"                                

#### 7.3.1.6  FEhist Reduccion dimensionalidad con canaritos

Esta etapa solo se mostrará a la *modalidad Anlista Sr* por algun canal secreto de forma de no confundir a los *Analista Jr*  nni distraer con detalles operativos a la estratégica *Modalidad Gerencial*

In [16]:
# No se implementa la reduccion de la dimensionalidad con canaritos

### 7.3.2 Modelado

#### 7.3.2.1 Training Strategy

Esta etapa de Workflow de  Training Strategy esta pensada para la *Modalidad Gerencial* que posee el dataset de [202005, 202109]
<br> Si usted es un Analista, posee el periodo de [201901, 202109] y deberá experimentar en que meses le conviene experimentar

<br> A la *Modalidad Gerencial* no se le complicada la vida con el undersampling de los continua, por eso PARAM$trainingstrategy$training_pct <- 1.0
<br> Sin embargo, si usted es  *Analista SR* posee un dataset 50 veces ( filas x columnas) más grande que la *Modalidad Gerencial*  y por un tema de velocidad y experimentación más rápida puede llegar a necesitar activar el undersampling de la clase mayoritaria, a pesar de estar corriendo en Google Cloud.

Se hace una estrategia de entrenamiento muy sencilla, tomando todos los meses posibles, SIN eliminar nada x pandemia ni por ningun otro motivo

* future = 202109  obviamente completo

* final_train =  [ 202005, 202107 ]  con un undersampling de los CONTINUA del 2% por un tema de velocidad

* training
   * testing = NO HAY
   * validation =  202107   completo, sin undersampling
   * training = [ 202005, 202106 ]  donde se consideran el 100% de los CONTINUA

In [17]:
meses <- unique(dataset$foto_mes)

meses_training <- meses[1:26]

meses_final_train <- meses[1:28]

In [18]:
meses_training

[1] 201901 201902 201903 201904 201905 201906 201907 201908 201909 201910
[11] 201911 201912 202001 202002 202003 202004 202005 202006 202007 202008
[21] 202009 202010 202011 202012 202101 202102

In [19]:
PARAM$trainingstrategy$validate <- c(202104, 202105)

PARAM$trainingstrategy$training <- meses_training

PARAM$trainingstrategy$training_pct <- .1


PARAM$trainingstrategy$positivos <- c( "BAJA+1", "BAJA+2")

In [20]:
# seteo la clase01   1={BAJA+1, BAJA+2}   0={CONTINUA}
dataset[, clase01 := ifelse( clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0 )]

In [21]:
# los campos en los que se entrena
campos_buenos <- copy( setdiff(
    colnames(dataset), c("clase_ternaria","clase01","azar"))
)

In [22]:
# preparo para que se puede hacer undersampling de los CONTINUA
#  solamente por un tema de VELOCIDAD
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar:=runif(nrow(dataset))]

# undersampling de los CONTINUA
dataset[, fold_train :=  foto_mes %in%  PARAM$trainingstrategy$training &
    (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
     azar < PARAM$trainingstrategy$training_pct ) ]


if( !require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

dtrain <- lgb.Dataset(
  data= data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label= dataset[fold_train == TRUE, clase01],
  free_raw_data= TRUE
)

Loading required package: lightgbm



In [23]:
# datos de validation
dvalidate <- lgb.Dataset(
  data= data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label= dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data= TRUE
)

nrow(dvalidate)

[1] 327335

####  7.3.2.2. Hyperparameter Tuning

* Clase binaria que se optimiza :  positivos = [ BAJA+1, BAJA+2 ]

* Metrica que se optimiza **AUC** Area Under Curve de la  ROC Curve

es muy importante notar que intencionalmente  **NO** se está optimizando la funcion de ganancia del problema

* Cantidad de iteraciones inteligentes de la Optimizacion Bayesiana = **10**

* Parametros no default, fijos de LightGBM que no se optimizan
  * max_bin = 31 , Alienigenas Ancestrales contruyeron las pirámides y dejaron a la humanidad en un jeroglifico  *max_bin=31*
  * feature_fraction = 0.5  para poner algo que generalmente no falla
  * learning_rate = 0.03  para que aprenda lento


* Parametros que se optimizan en la Bayesian Optimization
  * num_leaves  [8, 256]
  * min_data_in_leaf  [8, 8192]

In [24]:
# paquetes necesarios para la Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package:R.utils’:

    resample, setThreshold


Loading required package: smoof

Loading required package: checkmate


Attaching package: ‘checkmate’


The following object is masked from ‘package:DiceKriging’:

    checkNames


The following object is masked from ‘package:R.utils’:

    asInt



Attaching package: ‘smoof’


The following objects are masked from ‘package:R.oo’:

    getDescription, getName




Definición de la Bayesian Optimization
<br> Si se desea optimizar un hiperparámetro que esta como fijo, debe QUITARSE de param_fijos y agregarse a PARAM$hipeparametertuning$hs

In [25]:
# valor ridiculamente bajo para que corra rapido en el aula y no molestar a la *Modalidad Gerencial*
PARAM$hipeparametertuning$num_interations <- 50

# parametros fijos del LightGBM
PARAM$lgbm$param_fijos <- list(
  objective= "binary",
  metric= "auc",
  first_metric_only= TRUE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  verbosity= -100,
  force_row_wise= TRUE, # para evitar warning
  seed= PARAM$semilla_primigenia,
  max_bin= 31,
  learning_rate= 0.03,
  feature_fraction= 0.5,
  num_iterations= 2048,  # valor grande, lo limita early_stopping_rounds
  early_stopping_rounds= 200
)

PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 2L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 2L, upper = 8192L)
)

Función "señora caja negra"  que es llamada para verificar la realidad por la Bayesian Optimization

In [26]:
# En  x llegan los parmaetros de la bayesiana
#  devuelve la AUC en validate del modelo entrenado
#  en el parametro x llegan los hiperparámetros que se estan optimizando

EstimarGanancia_AUC_lightgbm <- function(x) {

  message(format(Sys.time(), "%a %b %d %X %Y"))

  # uno la lista de hiperparametros : fijos + variables
  param_completo <- c(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelo_train <- lgb.train(
    data= dtrain,
    valids= list(valid = dvalidate),
    eval= "auc",
    param= param_completo,
    verbose= -100
  )

  # recupero la AUC en validation
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # esta es la forma de devolver un parametro extra
  attr(AUC, "extras") <- list("num_iterations"= modelo_train$best_iter)

  # hago espacio en la memoria
  rm(modelo_train)
  gc(full= TRUE, verbose= FALSE)

  return(AUC)
}

seteo de la Bayesian Optimization (complejo)
<br> copiado y pegado de la documentación de la librería

In [27]:
configureMlr(show.learner.output = FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo
obj.fun <- makeSingleObjectiveFunction(
    fn= EstimarGanancia_AUC_lightgbm, # la funcion que voy a maximizar
    minimize= FALSE, # estoy Maximizando AUC
    noisy= FALSE,
    par.set= PARAM$hipeparametertuning$hs,
    has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
    save.on.disk.at.time= 600,
    save.file.path= "HT.RDATA"
)

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
    ctrl,
    iters= PARAM$hipeparametertuning$num_interations  # cantidad de iteraciones inteligentes
)

# defino el método estandar para la creacion de los puntos iniciales
#   los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

# mas configuraciones
surr.km <- makeLearner(
    "regr.km",
    predict.type= "se",
    covtype= "matern3_2",
    control= list(trace = TRUE)
)

Corrida de la Bayesian Optimization,  aqui se hace el trabajo pesado
<br> por favor no se asuste con los warnings que pudieran aparecer

Si corrío a medias y llegó a las iteraciones inteligentes, en el archivo binario HT.RDATA quedó lo ya procesado y es utilizado para retomar la corrida desde lo último que llegó a grabar.

In [28]:
# inicio la optimizacion bayesiana

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue("HT.RDATA") # retomo en caso que ya exista
}

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 969 points instead of 1000!”
Mon Jun 30 11:26:20 2025

[mbo] 33: num_leaves=256; min_data_in_leaf=4885 : y = 0.933 : 528.3 secs : infill_ei

Saved the current state after iteration 34 in the file HT.RDATA.

Mon Jun 30 11:35:11 2025

[mbo] 34: num_leaves=219; min_data_in_leaf=2410 : y = 0.933 : 393.7 secs : infill_ei

Mon Jun 30 11:41:45 2025

[mbo] 35: num_leaves=85; min_data_in_leaf=5638 : y = 0.933 : 497.5 secs : infill_ei

Saved the current state after iteration 36 in the file HT.RDATA.

Mon Jun 30 11:50:04 2025

[mbo] 36: num_leaves=249; min_data_in_leaf=5416 : y = 0.933 : 533.9 secs : infill_ei

Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 663 points instead of 1000!”
Warning message in generateDesign(control$infill.opt.focussearch.points, ps.local, :
“generateDesign could only produce 629

la bayesian optimization ha corrido, extraigo los mejores hiperparametros

In [29]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y, -num_iterations)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file="BO_log.txt",
  sep="\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  list(num_leaves, min_data_in_leaf, num_iterations)
]

print(PARAM$out$lgbm$mejores_hiperparametros)

   num_leaves min_data_in_leaf num_iterations
        <int>            <int>          <int>
1:        256             5310           1001


### 7.3.3 Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la optimización bayesiana

In [30]:
PARAM$trainingstrategy$final_train <- meses_final_train #meses_final_train

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train ]

# creo el dfinal_train en formato  LightGBM
dfinal_train <- lgb.Dataset(
  data= data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with= FALSE]),
  label= dataset[fold_final_train == TRUE, clase01],
  free_raw_data= TRUE
)

nrow( dfinal_train) # verifico el tamaño

[1] 4060310

#### Final Training Hyperparameters

In [31]:
# uno los parametros fijos y los mejores encontrados de los variables
fijos <- copy(PARAM$lgbm$param_fijos)

# quito lo que optimice en la Bayesian Optimization
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

In [ ]:
PARAM$trainingstrategy$future <- c(202106, 202107)  # c(202109)

for (semilla in c(277073, 470779, 314159, 358979, 589793)){
    fijos$seed = semilla

    # agrego a los hiperparametros fijos los que encontre con la Bayesian Optimization
    param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)
    # Entrenamiento del modelo
    final_model <- lgb.train(
      data= dfinal_train,
      param= param_final,
      verbose= -100
    )

    # grabo a disco el modelo en un formato para seres humanos ... ponele ...
    lgb.save(final_model, paste0("modelo_", semilla, ".txt"))

    # ahora imprimo la importancia de variables
    tb_importancia <- as.data.table(lgb.importance(final_model))
    archivo_importancia <- paste0("impo_", semilla, ".txt")

    fwrite( tb_importancia,
      file= archivo_importancia,
      sep= "\t"
    )

    #Predicción para datos del futuro
    dfuture <- dataset[ foto_mes %in% PARAM$trainingstrategy$future ]    

    prediccion <- predict(
      final_model,
      data.matrix(dfuture[, campos_buenos, with= FALSE])
    )

    tb_prediccion <- dfuture[, list(numero_de_cliente, clase_ternaria)]
    tb_prediccion[, prob := prediccion]

    # grabo las probabilidad del modelo
    #  me va a ser util para hacer Ensembles de modelos
    fwrite(tb_prediccion,
      file= paste0("prediccion_", semilla, ".txt"),
      sep= "\t"
    )

    # Elimino cosas de la memoria por las dudas
    rm(final_model)
    rm(tb_importancia)
    rm(dfuture)
    rm(prediccion)
    rm(tb_prediccion)
    
}

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")